# Creating the Commitment Transaction with in-flight htlc

In this section, we'll build a lightning channel commitment transaction with in-flight htlc from scratch using Python. We'll walk through each part of the transaction — how it's constructed and signed. The process will be tested using Bitcoin Core in regtest mode.

## Setup

For this notebook, we’ll use the basepoints derivated in the `chapter 0 - lightning node keys derivation` the funding transaction created in the `chapter 1 - channel funding transaction` and parts of the first commitment transaction created in the `chapter 2 - initial commitment transaction`.

In [1]:
%run "../chapter 2 - initial commitment transaction/initial commitment transaction.ipynb"

2026-02-13T19:13:46.919000Z TestFramework (INFO): PRNG seed is: 7866334107292442361
2026-02-13T19:13:46.921000Z TestFramework (INFO): Initializing test directory /tmp/bitcoin_func_test_h_i1xdln
🟢 New TestShell started. Block height: 0
Alice Per Commitment Seed 34b581ec20bf2c6cae3d4d4dcbfddc8a3727a1e9a57c55f3520e770607898c06
Bob Per Commitment Seed 89c994b3ddad4698acee71e42d8bcace48eea739caaba371eb110e77663ec56d
Alice Payment BasePoint:  025f892a06124391e2f38ce35d943cdc09f63e203330dbd9cb6113a903e0738458
Bob Payment BasePoint:  02f98efd3f2b2fbe7bd83c419f5f64f8280798b8a9175fdb77c0091bbb95c79506
To obscure commitment number 0xb433fd43a66f
Alice funding pubkey: 2fac0d7dd9f1771872f72771dedcca3ceec45cac9a155b6ba1e7949a7268568c
Alice funding privkey: 6d8470a984622a1ec4eee9a24de0c4e4e9a314fc78bff0305c4742e61d5117b3
Alice funding address: bcrt1p97kq6lwe79m3suhhyacaahx28nhvgh9vng24k6apu72f5ung26xqqzpks8
Alice sweeper pubkey: 52bbd24b405772a7cfda12e8093ca663b9d461a385959e1f30181ee975606181
Alice s

## HTLC (Hashed Time Locked Contracts)

Once the channel is established, it can be used to make payments via HTLCs (Hashed Time-Locked Contracts).

Updates to the channel state are sent in batches: one or more `update_ messages` are exchanged before a `commitment_signed` message is sent. In the diagram below, we show only one`update_message` per `commitment_signed` for simplicity:

    +-----------+                                                      +-----------+
    |           |                                                      |           |
    |           |------ channel_reestablished (next local nonce) ----->|           | 
    |           |<----- channel_reestablished (next local nonce) ------|           |
    |   Alice   |                                                      |    Bob    |
    |           |                                                      |           |    
    |           |----------------- update_add_htlc ------------------->|           |
    |           |----- commitment_signed (partial sig) (jit nonce) --->|           |
    |           |<--------- revoke_and_ack (next local nonce) ---------|           |
    |           |<---- commitment_signed  (partial sig) (jit nonce) ---|           |
    |           |---------- revoke_and_ack (next local nonce) -------->|           |
    |           |                                                      |           |
    |           |                                                      |           |
    +-----------+                                                      +-----------+

The messages are defined in the [BOLT 2](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#adding-an-htlc-update_add_htlc).

### The `update_add_htlc` Message

Alice sends the `update_add_htlc`  message to Bob, which includes the details of the HTLC being offered:
- channel_id
- id (htlc counter starting in zero)
- amount_msat
- payment_hash
- cltv_expiry
- onion_routing_packet

### The `commitment_signed` Message

Shortly after sending the `update_add_htlc` message, Alice commits to the new channel state so that the HTLC can be safely included by Bob. At this point, Bob has the HTLC information and has constructed a new commitment transaction, but he hasn't yet received Alice's signature for it.

Alice sends a `commitment_signed`message to Bob, which includes her signature for the new commitment transaction as well as the HTLC output it contains:
- channel_id
- signature
- num_htlcs
- partial_signature_with_nonce

### The `revoke_and_ack` Message

Now that Bob has a new signed commitment, he needs to acknowledge it and revoke the previous one. He does this by sending a`revoke_and_ack` message to Alice:

- channel_id
- per_commitment_secret
- next_per_commitment_point
- nonce

Finally, Bob sends a `commitment_signed` message to Alice, and Alice responds with a `revoke_and_ack` message. At this point, both parties have fully signed the new commitment transaction.


### The Unsigned Transaction

### The Input

The input is the channel funding transaction.

In [2]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER (new to segwit)
marker = bytes.fromhex("00")

# FLAG (new to segwit)
flag = bytes.fromhex("01")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
txid = (bytes.fromhex(funding_channel_txid))[::-1]
funding_channel_index = 0
index = funding_channel_index.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# sequence: upper 8 bits are 0x80, lower 24 bits are the upper 24 bits of the obscured commitment number
# Commitment number of the second commitment transaction.
commitment_number = 1
# obscured commitment number is result of xor operation 
commitment_number_obscured = lower48_to_obscure ^ commitment_number
# Extract the upper 24 bits of the obscured commitment number
upper_24_bits = (commitment_number_obscured >> 24) & 0xFFFFFF
# Combine the upper 8 bits (0x80) with the lower 24 bits (upper 24 of obscured number)
sequence = (0x80 << 24) | upper_24_bits
# Convert to bytes (byte, big-endian)
sequence = sequence.to_bytes(4, byteorder='big')
# Convert to little-endian 
sequence = sequence[::-1]

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

print("Commitment Number Osbscured", hex(commitment_number_obscured))

Commitment Number Osbscured 0xb433fd43a66e


### The Outputs

Let’s assume Alice is sending 0.5 million satoshis to Bob, for this reason Alice node is offering an HTLC to Bob's node of this value.

For this alice commitment transaction we must add an offered HTLC otuput and to remote anchor output.

#### The Offered HTLC otuput

The offered HTLC otuput pays out to the remote party if they reveal the pre-image to a given hash before a certain CLTV timeout. After the timeout, the local party will be able to claim the output via the htlc-timeout transaction. If the commitment transaction is a revoked state, then the remote party should be able to sweep the output at any time.



    +------+---------------+
    | OP_1 |       Q       |
    +------+---------------+
                   ^
                   |   +-------------------+
                    ---| P(revocation) + T |
                       +-------------------+
                                         ^
                                         |
                                   +-----------+        
                                   | T = t * G |
                                   +-----------+        
                                         ^
                                         |
     +---+   +-------------------------------------------------------+
     | t | = | TaggedHash ("Taptweak", P(revocation) || script_root) |
     +---+   +-------------------------------------------------------+
                                                             ^
                                                             |
                                                          +-----+
                          ------------------------------> | hAB |<------------------
                         |                                +-----+                   |
                         |                                                          |
                         |                                                          |
                      +----+                                                     +----+
                      | hA |                                                     | hB |
                      +----+                                                     +----+
                         ^                                                          ^
                         |                                                          |
      +---------------------------------+                                           |
      | P(local_htlc) OP_CHECKSIGVERIFY |                                           |
      |                                 |                                           |
      | P(remote_htlc) OP_CHECKSIG      |                                           |
      +---------------------------------+                                           |
      +-------------------------------------------------------------------------------+ 
      | OP_SIZE 32 OP_EQUALVERIFY OP_HASH160 <RIPEMD160(payment_hash)> OP_EQUALVERIFY |
      |                                                                               |
      | P(remote_htlc) OP_CHECKSIG OP_1 OP_CHECKSEQUENCEVERIFY OP_DROP                |
      +-------------------------------------------------------------------------------+

#### The Remote Anchor Output

This is the output that the remote party, will be able to use to CPFP the commitment transaction and it is spendable by anyone after 16 blocks.

    +------+---------------+
    | OP_1 |       Q       |
    +------+---------------+
                   ^
                   |   
         +---------------+
         | P(remote) + T |
         +---------------+
                       ^
                       |  
                 +-----------+
                 | T = t * G |
                 +-----------+     
                      ^                          
                      |
                    +---+   +---------------------------------------------------+
                    | t | = | TaggedHash ("Taptweak", P(remote) || script_root) |
                    +---+   +---------------------------------------------------+
                                                                           ^
                                                                           |
                                                                         +---+
                                                                         | h |
                                                                         +---+
                                                                           ^
                                                                           |
                                                                   +--------------+
                                                                   | OP_16 OP_CSV |
                                                                   +--------------+





In [3]:
# Create Alice per-commitment
alice_per_commitment = per_commitment(alice_per_commitment_seed, commitment_number)
# Create Bob per-commitment
bob_per_commitment = per_commitment(bob_per_commitment_seed, commitment_number)

# Create Alice Delayed Public Key
alice_delayed_pubkey = derivate_key(alice_node_seed, family=4, channel_index=0).get_pubkey(alice_per_commitment.get_pub())
print(f"Alice Delayed PubKey: {alice_delayed_pubkey.get_bytes(bip340=True).hex()}")

# Create Bob Local Public Key which in this case is the P(remote) for Alice
bob_payment_pubkey = bob_payment.get_pubkey(bob_per_commitment.get_pub())
print(f"Bob Payment PubKey: {bob_payment_pubkey.get_bytes(bip340=True).hex()}")

# Create Bob Revocation Public Key
bob_revocation_pubkey = derivate_revocation_key(bob_node_seed, channel_index=0).get_pubkey(alice_per_commitment.get_pub())
print(f"Bob Revocation PubKey: {bob_revocation_pubkey.get_bytes(bip340=True).hex()}")

# Create Alice HTLC Public Key
alice_htlc_pubkey = derivate_key(alice_node_seed, family=2, channel_index=0).get_pubkey(alice_per_commitment.get_pub())
print(f"Alice HTLC PubKey: {alice_htlc_pubkey.get_bytes(bip340=True).hex()}")

# Create Bob HTLC Public Key
bob_htlc_pubkey = derivate_key(bob_node_seed, family=2, channel_index=0).get_pubkey(bob_per_commitment.get_pub())
print(f"Bob HTLC PubKey: {bob_htlc_pubkey.get_bytes(bip340=True).hex()}")

# Outputs for Alice Second Commitment Transaction
# 0x04 outputs
output_count = bytes.fromhex("04")

# ANCHOR OUTPUT
# Anchor output amount
anchor_output_value_satoshis = 330
anchor_output_value = anchor_output_value_satoshis.to_bytes(8, byteorder="little", signed=False)

# OP_16 OP_CSV
script = CScript([OP_16, OP_CHECKSEQUENCEVERIFY])

# Method: ser_string(data) is a function which adds compactsize to input data.
hash_input = TAPSCRIPT_VER + ser_string(script)

# Anchor Output script_root
script_root = tagged_hash("TapBranch", hash_input)

# Alice Anchor Output Tagged Hash
taptweak = tagged_hash("TapTweak", alice_delayed_pubkey.get_bytes(bip340=True) + script_root)
alice_delayed_pubkey_tweaked = alice_delayed_pubkey.tweak_add(taptweak)
# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
alice_anchor_spk = bytes.fromhex("51") + varint_len(alice_delayed_pubkey_tweaked.get_bytes(bip340=True)) + alice_delayed_pubkey_tweaked.get_bytes(bip340=True)

# Bob Anchor Output Tagged Hash
taptweak = tagged_hash("TapTweak", bob_payment_pubkey.get_bytes(bip340=True) + script_root)
bob_payment_pubkey_tweaked = bob_payment_pubkey.tweak_add(taptweak)
# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
bob_anchor_spk = bytes.fromhex("51") + varint_len(bob_payment_pubkey_tweaked.get_bytes(bip340=True)) + bob_payment_pubkey_tweaked.get_bytes(bip340=True)

# Sort by scriptPubKey (lexicographic order)
anchors = [
    (alice_anchor_spk, "alice_anchor"),
    (bob_anchor_spk,   "bob_anchor"),
]

anchors_sorted = sorted(anchors, key=lambda x: x[0])

# OFFERED HTLC OUTPUT
# HTLC output amount
htlc_output_value_satoshis = 500000
htlc_output_value = htlc_output_value_satoshis.to_bytes(8, byteorder="little", signed=False)

# Create the leaf scripts A and B
# Leaf script A: P(local_htlc) OP_CHECKSIGVERIFY P(remote_htlc) OP_CHECKSIG
scriptA = CScript([alice_htlc_pubkey.get_bytes(bip340=True), OP_CHECKSIGVERIFY, bob_htlc_pubkey.get_bytes(bip340=True), OP_CHECKSIG])

# Leaf script B: OP_SIZE 32 OP_EQUALVERIFY OP_HASH160 <RIPEMD160(payment_hash)> OP_EQUALVERIFY P(remote_htlc) OP_CHECKSIG OP_1 OP_CHECKSEQUENCEVERIFY OP_DROP

# We use a dummy payment_hash here. In a real scenario, this would be the hash of the preimage for the HTLC.
# And would be sent in the message update_add_htlc.
sha = sha256(b'payment').digest()
payment_hash = hashlib.new("ripemd160", sha).digest()

scriptB = CScript([OP_SIZE, bytes.fromhex("20"), OP_EQUALVERIFY, OP_HASH160, payment_hash, OP_EQUALVERIFY, bob_htlc_pubkey.get_bytes(bip340=True), OP_CHECKSIG, OP_1, OP_CHECKSEQUENCEVERIFY, OP_DROP])

# Compute TapLeaves A and B
# Method: ser_string(data) is a function which adds compactsize to input data.
hash_inputA = TAPSCRIPT_VER + ser_string(scriptA)
hash_inputB = TAPSCRIPT_VER + ser_string(scriptB)
htlc_taggedhash_leafA = tagged_hash("TapLeaf", hash_inputA)
htlc_taggedhash_leafB = tagged_hash("TapLeaf", hash_inputB)

print("\ntapleaf_hash:", htlc_taggedhash_leafA.hex())
print("\ntapleaf_hash:", htlc_taggedhash_leafB.hex())

# Method: Returns tapbranch hash. Child hashes are lexographically sorted and then concatenated.
# l: tagged hash of left child
# r: tagged hash of right child
def tapbranch_hash(l, r):
    return tagged_hash("TapBranch", b''.join(sorted([l,r])))

# Compute Internal node TapBranch AB
root_ab = tapbranch_hash(htlc_taggedhash_leafA, htlc_taggedhash_leafB)
print("\ntapbranch_hash (root_ab):", root_ab.hex())

# Compute TapTweak
taptweak = tagged_hash("TapTweak", bob_revocation_pubkey.get_bytes(bip340=True) + root_ab)
bob_revocation_pubkey_tweaked = bob_revocation_pubkey.tweak_add(taptweak)
print("\ntaptweak:", taptweak.hex())

# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
alice_offered_htlc_spk = bytes.fromhex("51") + varint_len(bob_revocation_pubkey_tweaked.get_bytes(bip340=True)) + bob_revocation_pubkey_tweaked.get_bytes(bip340=True)

Alice Delayed PubKey: dc5cd4cbc27d9856a98bdf5ab9da85023f20c960c52c93b7e36d40a04b5cfbe5
Bob Payment PubKey: 8f16659036b06b785dd82ec3ac4d5518c2c11e2e101612e6dc983bfeb6dc7ba5
Bob Revocation PubKey: c39a6b0cffa569f243cec8252a1e5f93b4b072247ff18773917b72fb6341f608
Alice HTLC PubKey: 7546c6ab3a8c005ce24e9feb7154fd35b62ac0f9125beb5e692d4d912a95de77
Bob HTLC PubKey: 7bd8383df06640c0efe760bb2bac1a1585bdc664e6f1cb61c41a76a6f2be3484

tapleaf_hash: 509fbe809f2233d8daa3bca96c2735a7f7a1bd75eb48976978029ac95cd0fd13

tapleaf_hash: 8da58f2aee0bccdd60c6065603a5d2ab655e79971b068228afbccbb02220f54b

tapbranch_hash (root_ab): 68c39f1490cf85a839045dec1faa2f840f29aabf8b389057b5c49c1ec3dcdaff

taptweak: 0c925b61e6f3c4ec48b69171ab350036b942f775f3edf265425395cda90529ea


### Fee Calculation

In [4]:
# Second commitment expected weight 
commitment_weight = taproot_commit_weight(NumTapOut=2, NumAnchorOut=2)
print("Commitment Transaction Expected Weight:", commitment_weight)

# From open_channel and update_feerate messages (1 sat/vb = 250 sat/kw)
feerate_per_kw = 250

# Calculate the fee for the second commitment transaction
commitment_fee = int(commitment_weight * feerate_per_kw / 1000)

Commitment Transaction Expected Weight: 968


In [5]:
# TO LOCAL OUTPUT
# to local output amount
to_local_value_sat = channel_value_sat - anchor_output_value_satoshis * 2 - htlc_output_value_satoshis - commitment_fee
to_local_value = to_local_value_sat.to_bytes(8, byteorder="little", signed=False)

# Create the leaf scripts A and B
# P(local_delayed) OP_CHECKSIG to_self_delay OP_CSV OP_DROP
to_self_delay = 144
scriptA = CScript([alice_delayed_pubkey.get_bytes(bip340=True), OP_CHECKSIG, to_self_delay.to_bytes(2,"big"), OP_CHECKSEQUENCEVERIFY, OP_DROP])

# P(local_delayed) OP_DROP P(revocation) OP_CHECKSIG
scriptB = CScript([alice_delayed_pubkey.get_bytes(bip340=True), OP_DROP, bob_revocation_pubkey.get_bytes(bip340=True), OP_CHECKSIG])

# Compute TapLeaves A and B
# Method: ser_string(data) is a function which adds compactsize to input data.
hash_inputA = TAPSCRIPT_VER + ser_string(scriptA)
hash_inputB = TAPSCRIPT_VER + ser_string(scriptB)
taggedhash_leafA = tagged_hash("TapLeaf", hash_inputA)
taggedhash_leafB = tagged_hash("TapLeaf", hash_inputB)

# Method: Returns tapbranch hash. Child hashes are lexographically sorted and then concatenated.
# l: tagged hash of left child
# r: tagged hash of right child
def tapbranch_hash(l, r):
    return tagged_hash("TapBranch", b''.join(sorted([l,r])))

# Compute Internal node TapBranch AB
root_ab = tapbranch_hash(taggedhash_leafA, taggedhash_leafB)

# Compute TapTweak
taptweak = tagged_hash("TapTweak", NUMS.get_bytes(bip340=True) + root_ab)
NUMS_tweaked = NUMS.tweak_add(taptweak)

# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
alice_to_local_spk = bytes.fromhex("51") + varint_len(NUMS_tweaked.get_bytes(bip340=True)) + NUMS_tweaked.get_bytes(bip340=True)

outputs = (
    anchor_output_value
    + varint_len(anchors_sorted[0][0])
    + anchors_sorted[0][0]
    + anchor_output_value
    + varint_len(anchors_sorted[1][0])
    + anchors_sorted[1][0]
    + to_local_value
    + varint_len(alice_to_local_spk)
    + alice_to_local_spk
    + htlc_output_value
    + varint_len(alice_offered_htlc_spk)
    + alice_offered_htlc_spk
)

# Locktime: upper 8 bits are 0x20, lower 24 bits are the lower 24 bits of the obscured commitment number
# Extract the lower 24 bits of the obscured commitment number
lower_24_bits = commitment_number_obscured & 0xFFFFFF
# Combine the upper 8 bits (0x20) with the lower 24 bits (lower 24 of obscured number)
locktime = (0x20 << 24) | lower_24_bits
# Convert to bytes (1 byte, big-endian)
locktime = locktime.to_bytes(4, byteorder='big')
locktime = locktime[::-1]

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)
print("unsigned_tx: ", unsigned_tx.hex())

# Decode the unsigned transaction to verify it looks correct
decoded = node.decoderawtransaction(unsigned_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

unsigned_tx:  020000000137e5c5982ce6833e4b032ce0e8d0029d968fd1bdda335f17e5a7f87dbe6df6730000000000fd33b480044a010000000000002251200a0223e7992877974ec2a63c30d829c698de2ebf37b56b3372e22ab6d69311544a0100000000000022512081846514fd9471e5b9a961cb94b0d6a54036f9c1e81216f858b86bbf6efb74009a9d0700000000002251200ad08ce23ebfe765c0749f63a3a78efed80f4f7cfbac25ef544f05c2a07a009d20a1070000000000225120ba9dfd92bdd3bb7b3b4faa510e1076c32c5105c0f0e620d238486e8fa84673836ea64320
{
  "txid": "e0104ef28d86323e986216435398a6c33cf8922781d35bf32fa3d85c20c7fdf8",
  "hash": "e0104ef28d86323e986216435398a6c33cf8922781d35bf32fa3d85c20c7fdf8",
  "version": 2,
  "size": 223,
  "vsize": 223,
  "weight": 892,
  "locktime": 541304430,
  "vin": [
    {
      "txid": "73f66dbe7df8a7e5175f33dabdd18f969d02d0e8e02c034b3e83e62c98c5e537",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "sequence": 2159293437
    }
  ],
  "vout": [
    {
      "value": "0.00000330",
      "n": 0,
      "s

## The sighash for the key path spend

This is the “Schnorr key spend” case: you’re proving knowledge of the (tweaked) internal private key, with no script branch revealed. The message preimage is called msg_digest in [BIP-341](https://github.com/bitcoin/bips/blob/master/bip-0341.mediawiki).

In [6]:
index_of_this_input = bytes.fromhex("0000 0000")

# SIGHASH for key path spend
# See BIP-341 for details
sighash_epoch = bytes.fromhex("00")

# Control
hash_type = bytes.fromhex("00") # SIGHASH_DEFAULT (a new sighash type meaning implied SIGHASH_ALL)

# Transaction data
sha_prevouts = sha256(txid + index).digest()

sha_amounts = sha256(channel_value).digest()

# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
sha_scriptpubkeys = sha256(
    varint_len(channel_spk)
    + channel_spk
).digest()

sha_sequences = sha256(sequence).digest()

sha_outputs = sha256(outputs).digest()

# Data about this input
spend_type = bytes.fromhex("00") # no annex present

sig_msg = (
    sighash_epoch
    + hash_type
    + version
    + locktime
    + sha_prevouts
    + sha_amounts
    + sha_scriptpubkeys
    + sha_sequences
    + sha_outputs
    + spend_type
    + index_of_this_input
)

tag_hash = sha256("TapSighash".encode()).digest()
sighash = sha256(tag_hash + tag_hash + sig_msg).digest()


## Signing the sighash

In [7]:
# Build participants to sort them using the same rule used in pubkeys aggregation.
participants = [
    (pubkey_alice_musig2.get_bytes(bip340=False), privkey_alice_musig2.get_bytes(), alice_per_commitment_seed),
    (pubkey_bob_musig2.get_bytes(bip340=False),   privkey_bob_musig2.get_bytes(), bob_per_commitment_seed),
]

# Reorder participants to match the sorted pubkeys
pk_to_tuple = {pk: (pk, sk, pcs) for pk, sk, pcs in participants}
participants = [pk_to_tuple[pk] for pk in sorted_pubkeys]

# Alice and Bob generates its own nonce pair (secnonce, pubnonce)
secnonces, pubnonces = [], []
for pk, sk, pcs in participants:
    # Use per-commitment nonce for Alice to get deterministic results
    if pk == pubkey_alice_musig2.get_bytes(bip340=False):
        sec, pub = nonce_per_commitment(pcs, commitment_number, sk, pk, agg_pubkey_tweaked, sighash)
    else:
        sec, pub = nonce_gen(sk, pk, agg_pubkey_tweaked, sighash, None)
    secnonces.append(sec)
    pubnonces.append(pub)

# Alice and Bob has exchanged already the pubnonces, so they can agregate them and create the session context
agg_nonce = nonce_agg(pubnonces)
session = SessionContext(agg_nonce, sorted_pubkeys, [bip86_tweak], [True], sighash)

# Alice and Bob produces their partial signatures
psigs = [sign(sec, sk, session) for sec, (_, sk, _) in zip(secnonces, participants)]

# Each side verify the other’s partial signature before proceeding
for i, psig in enumerate(psigs):
    assert partial_sig_verify(psig, pubnonces, sorted_pubkeys, [bip86_tweak], [True], sighash, i)

# Each side combines partial signatures into the final Schnorr signature
agg_sig = partial_sig_agg(psigs, session)

# Sanity check: verify with BIP340 against the *tweaked* x-only key ---
ok = schnorr_verify(sighash, agg_pubkey_tweaked, agg_sig)
print("aggregated Schnorr verifies?", ok)

aggregated Schnorr verifies? True


## The signed transaction

In [8]:
witness = (
    bytes.fromhex("01") # one stack item in the witness
    + varint_len(agg_sig)
    + agg_sig
)

# the final signed transaction
signed_alice_commitment_tx = (
    version
    + marker
    + flag
    + input_count
    + inputs
    + output_count
    + outputs
    + witness
    + locktime
)

print("signed tx: ",signed_alice_commitment_tx.hex())
# Decode the unsigned transaction to verify it looks correct
decoded_alice_commitment_tx = node.decoderawtransaction(signed_alice_commitment_tx.hex())
print(json.dumps(decoded_alice_commitment_tx, indent=2, default=str))

print(node.testmempoolaccept(rawtxs=[signed_alice_commitment_tx.hex()]))

signed tx:  0200000000010137e5c5982ce6833e4b032ce0e8d0029d968fd1bdda335f17e5a7f87dbe6df6730000000000fd33b480044a010000000000002251200a0223e7992877974ec2a63c30d829c698de2ebf37b56b3372e22ab6d69311544a0100000000000022512081846514fd9471e5b9a961cb94b0d6a54036f9c1e81216f858b86bbf6efb74009a9d0700000000002251200ad08ce23ebfe765c0749f63a3a78efed80f4f7cfbac25ef544f05c2a07a009d20a1070000000000225120ba9dfd92bdd3bb7b3b4faa510e1076c32c5105c0f0e620d238486e8fa846738301403cca9503f21ac13c17ae615034635736ad49c067fab7d83e938287f647f4219a6f43a751c9427dd76a95ce42fee7a03ccbd92b7232a51e7dc98fd950e7452f4e6ea64320
{
  "txid": "e0104ef28d86323e986216435398a6c33cf8922781d35bf32fa3d85c20c7fdf8",
  "hash": "d71036ce600880b77f1e61e15a218e98d117591cbf279f9e8383397a0d4a2f5b",
  "version": 2,
  "size": 291,
  "vsize": 240,
  "weight": 960,
  "locktime": 541304430,
  "vin": [
    {
      "txid": "73f66dbe7df8a7e5175f33dabdd18f969d02d0e8e02c034b3e83e62c98c5e537",
      "vout": 0,
      "scriptSig": {
        "asm": "",
  

## Summary

This chapter demonstrated how HTLCs (Hashed Time-Locked Contracts) enable Lightning payments by adding conditional outputs to commitment transactions:

### Alice's Commitment Transaction (Offering HTLC)
- **Commitment number**: 1
- **Four outputs**:
  1. Alice's anchor (330 sats)
  2. Bob's anchor (330 sats)  
  3. Alice's to_local (499,098 sats)
  4. Offered HTLC (500,000 sats)

### Offered HTLC Output Structure
- **Internal key**: Bob's revocation_pubkey (allows penalty if revoked)
- **Two script paths**:
  1. **Path A - Success (Cooperative)**: `<alice_htlc> CHECKSIGVERIFY <bob_htlc> CHECKSIG`
  2. **Path B - Success (Bob claims with preimage)**:
     ```
     SIZE 32 EQUALVERIFY HASH160 <RIPEMD160(payment_hash)> EQUALVERIFY
     <bob_htlc> CHECKSIG 1 CHECKSEQUENCEVERIFY DROP
     ```

### HTLC Security Properties
- **Hash lock**: Payment requires preimage revealing
- **Time lock**: Sender can reclaim after timeout (CLTV)
- **Revocation**: Old HTLC states can be penalized
- **CSV delays**: Provide reaction time for disputes

### Fee Calculation
- Expected weight: 968 WU (with HTLC)
- Fee: 242 sats
- Outputs sorted by value (BOLT 3 requirement)

### Key Takeaway
HTLCs enable secure, conditional payments across the Lightning Network. The combination of hash locks (proof of payment) and time locks (refund mechanism) ensures that payments are atomic: either both parties receive what they expect, or the payment fails and funds return to sender. The Taproot structure provides privacy benefits by hiding unexecuted script paths.